# Tutorial Convert Solvency 2 XBRL-instances to CSV, HTML and pickles

This tutorial describes how to convert XBRL-instances to csv-, html- and pickle files per template.

We use Arelle, a open source package for processing XBRL. In addition this repository contains code to process the Solvency 2 and FTK instances efficiently.

In [ ]:
from arelle import ModelManager, Cntlr, ModelXbrl, XbrlConst, RenderingEvaluator, \
                   ViewFileRenderedGrid, ModelFormulaObject
from arelle import PackageManager, FileSource

import src
import pandas as pd
import os
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from datetime import datetime

## Initialize the Arelle model manager

First we specify the directories with the taxonomy and instances. You can put your own instances in the data/instances directory, or you can specify here the directories that you want to use.

In [ ]:
# the taxonomy should be data/taxonomy/arelle
# the instances you want to use should be in data/instances

XBRL_TAXONOMY_PATH = join('..', 'data', 'taxonomies')
XBRL_INSTANCES_PATH = join('..', 'data', 'instances')

LANGUAGE = "en-GB"
environ['XDG_CONFIG_HOME'] = XBRL_TAXONOMY_PATH

# The role defined in the model.xsd schema for resources representing codes of rows or columns is
euRCcode = 'http://www.eurofiling.info/xbrl/role/rc-code'

To process XBRL, we need a controller and a modelmanager object. 

In the controller you can specify logging. Here we have set logging to print in this notebook.

In [ ]:
# Now we make a modelmanager
# logFileName = "logToPrint" -> logging is print to notebook
# logFileName = "arelle.log" -> logging is to filename (use .json or .xml for specific format)

controller = Cntlr.Cntlr(logFileName = "logToPrint")
controller.webCache.workOffline = True
controller.logger.messageCodeFilter = None

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

## Initialize taxonomies

In [ ]:
taxonomies = ['FTK Taxonomy 2.1.0_tcm46-386386.zip', 
              'EIOPA_SolvencyII_XBRL_Taxonomy_2.4.0_with_external_hotfix.zip']
PackageManager.init(controller)
for taxonomy in taxonomies:
    fs = FileSource.openFileSource(filename = join(XBRL_TAXONOMY_PATH, taxonomy))
    fs.open()
    PackageManager.addPackage(controller, fs.baseurl)
    fs.close()
PackageManager.rebuildRemappings(controller)
PackageManager.save(controller)

## Read XBRL-instance in the modelmanager

Now we are able to read and process an XBRL-instance.

We read the example instances provided with the taxonomy.

In [ ]:
# Solvency 2:
prefix = ""
# the example instance of the quarterly templates for solo
instance_name = 'qrs_240_instance.xbrl'  
# the example instance of the annual templates
# instance_name = 'aeb_240_instance.xbrl'

# FTK:
# prefix = "FTK."
# the example instance of the FTK assets templates
# instance_name = 'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL'

In [ ]:
xbrl_instance = ModelXbrl.load(modelManager = modelmanager, 
                               url = join(XBRL_INSTANCES_PATH, instance_name))
RenderingEvaluator.init(xbrl_instance)

## Convert XBRL-instance to CSV and Pandas-pickle

For each template or table in the instance we export the results to a csv file and a Pandas pickle-file. 

A Pandas pickle-file maintains the correct indices, whereas the csv does not, so if you want to access the data read the pickle (we included an example below).

The csv-files and the pickle-files are stored in a subdirectory identical to the name of the XBRL-instance (without extension)

In [ ]:
# The location of the csv-files
subdir = basename(instance_name).split(".")[0]

In [ ]:
# get tables in instance and sort by short name and print the first ten tables
tables = list(xbrl_instance.modelRenderingTables)
tables.sort(key = lambda table: table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))
for table in tables:
    print(table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))

In [ ]:
# create csv and pickle files
# time_stamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

# use verbose_labels = False if you want the row-column code as column names
# use verbose_labels = True if you want labels as column names

for table in tables:
    obj = src.generateCSV.generateCSVTables(xbrl_instance, join(XBRL_INSTANCES_PATH, subdir), 
                                            table = table, 
                                            lang = LANGUAGE,
                                            verbose_labels = False)

In [ ]:
# construct one dataframe with all data from closed axis tables
df_closed_axis = pd.DataFrame()  
for table in tables:
    table_name = table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode)
    if exists(join(XBRL_INSTANCES_PATH, subdir, table_name + '.pickle')):
        df = pd.read_pickle(join(XBRL_INSTANCES_PATH, subdir, table_name + '.pickle'))  # read dataframe
        if df.index.nlevels == 2:  # if 2 indexes (entity, period) --> closed axis table
            if len(df_closed_axis) == 0:  
                # no data yet --> copy dataframe
                df_closed_axis = df.copy()
            else:  
                # join to existing dataframe
                df_closed_axis = df_closed_axis.join(df)
df_closed_axis.to_pickle(join(XBRL_INSTANCES_PATH, subdir, subdir + '.pickle'))

## Example to read a template from the pickle files

The easiest way to access the data of a separate template is to read the corresponding pickle-file.

In [ ]:
t = tables[12].genLabel(lang = LANGUAGE,strip = True, role = euRCcode)
df = pd.read_pickle(join(XBRL_INSTANCES_PATH, subdir, prefix + t + ".pickle"))
df

If you want to obtain the dataframe with all data of the templates with closed axes, then you can use:

In [ ]:
df = pd.read_pickle(join(XBRL_INSTANCES_PATH, subdir, subdir + ".pickle"))
df

## Validate instance from Arelle

It should be possible to validate the instance (performing the validation rules within the taxonomy) with Arelle with the following code. But we did not test this!

In [ ]:
# modelXbrl = modelmanager.load(join(XBRL_INSTANCES_PATH, instance_name))

In [ ]:
# controller = Cntlr.Cntlr(logFileName = "logToPrint")
# controller.webCache.workOffline = True
# controller.setLogCodeFilter(None)
# controller.logger.setLevel('INFO')

In [ ]:
# modelmanager.validateInferDecimals = True
# modelmanager.validateCalcLB = True
# modelmanager.validate()